# GIN Model for oral bioavailability dataset 

1. This notebook focuses on building a GIN model for oral bioavailabilty dataset using the GIN convolutional technique which is available on Pytorch Geometric 
2. The hyperparameters were found using the TPE algorithm using Optuna library after 30 trials 
3. Train/validate/test using 5-fold CV and process was repeated for 10 times.

### Note
1. Ensure that this notebook is in the same working directory as data folder, config.py, utils.py, engine.py and model.py
2. To load models, please download the saved models provided in google drive link from README.md
3. Comment away the training function to load saved models and reproduce results

In [1]:
from utils import seed_everything, LoadHOBDataset
from config import SEED_NO, NUM_FEATURES, NUM_GRAPHS_PER_BATCH, NUM_TARGET, DEVICE, PATIENCE, EPOCHS, N_SPLITS, params_gin
from engine import EngineHOB_no_edge
from model import GIN

import torch
import numpy as np
import optuna
from sklearn.model_selection import KFold
from torch_geometric.loader import DataLoader
import os 

## 2. Tuning of model (Suggest to skip and used the hyperparameters saved in config.py)
1. To tune the model, ensure data folder and this notebook are in the same working directory. Then, we make use of the Optuna library which allows us to use the Tree-structure Parzen Estimator Algorithm to find the best hyperparameters for us 
2. First, create a run_tuning function to include the training and validation step with early stopping mechansim 
3. Then, create an objective function of Optuna to evaluate the best hyperparameters in 30 trials. 

In [6]:
def run_tuning(train_loader, valid_loader, params):
    '''
    This function controls the tuning step of the model.

    Args:
    train_loader: Pytorch geometric DataLoader Class of train dataset
    valid_loader: Pytorch geometric DataLoader Class of validation dataset 
    params (dict): dictionary containing the hyperparameters (num_layers, hidden_size and learning_rate)
    '''
    model = GIN(num_features=NUM_FEATURES, num_targets=NUM_TARGET, num_layers=params['num_layers'], hidden_size=params['hidden_size'])
    model.to(DEVICE)
    optimizer=torch.optim.Adam(model.parameters(),lr = params['learning_rate'])
    eng = EngineHOB_no_edge(model, optimizer, device=DEVICE)

    best_loss = np.inf
    early_stopping_iter = PATIENCE
    early_stopping_counter = 0 

    for epoch in range(EPOCHS):
        train_loss = eng.train(train_loader)
        valid_loss = eng.validate(valid_loader)
        print(f'Epoch: {epoch+1}/{EPOCHS}, train loss : {train_loss}, validation loss : {valid_loss}')
        if valid_loss < best_loss:
            best_loss = valid_loss 
            early_stopping_counter=0 #reset counter
        else:
            early_stopping_counter +=1

        if early_stopping_counter > early_stopping_iter:
            print('Early stopping...')
            break
        print(f'Early stop counter: {early_stopping_counter}')
    
    return best_loss

In [6]:
def objective(trial):
    params = {
        'num_layers' : trial.suggest_int('num_layers', 1,3),
        'hidden_size' : trial.suggest_int('hidden_size', 64, 512),
        'learning_rate' : trial.suggest_float('learning_rate', 1e-3, 9e-3, log=True)
    }

    #load dataset 
    dataset_for_cv = LoadHOBDataset(root='./data/graph_data/data_oral_avail_train/', raw_filename='data_oral_avail_train_50.csv')
    kf = KFold(n_splits=N_SPLITS)
    fold_loss = 0

    for fold_no, (train_idx, valid_idx) in enumerate(kf.split(dataset_for_cv)):
        print(f'Fold {fold_no}')
        train_dataset= []
        valid_dataset = []
        for t_idx in train_idx:
            train_dataset.append(torch.load(f'./data/graph_data/data_oral_avail_train/processed/molecule_{t_idx}.pt'))
        for v_idx in valid_idx:
            valid_dataset.append(torch.load(f'./data/graph_data/data_oral_avail_train/processed/molecule_{v_idx}.pt'))

        seed_everything(SEED_NO)
        train_loader = DataLoader(train_dataset, batch_size=NUM_GRAPHS_PER_BATCH, shuffle=True)
        valid_loader = DataLoader(valid_dataset, batch_size=NUM_GRAPHS_PER_BATCH, shuffle=False)

        loss = run_tuning(train_loader, valid_loader, params)
        fold_loss += loss

    return fold_loss/5

study = optuna.create_study(direction = 'minimize')
study.optimize(objective, n_trials=30)
print(f'best trial:')
trial_ = study.best_trial
print(trial_.values)
print(f'Best parameters: {trial_.params}')

[I 2023-03-08 17:26:41,550] A new study created in memory with name: no-name-4b8d77f9-2fca-4a34-8d44-8289bb1bb401


Fold 0
Epoch: 1/300, train loss : 1.6848286837339401, validation loss : 0.6327188611030579
Early stop counter: 0
Epoch: 2/300, train loss : 0.7388698011636734, validation loss : 0.7309691309928894
Early stop counter: 1
Epoch: 3/300, train loss : 0.6848915070295334, validation loss : 0.6669426560401917
Early stop counter: 2
Epoch: 4/300, train loss : 0.6840784102678299, validation loss : 0.6986954808235168
Early stop counter: 3
Epoch: 5/300, train loss : 0.6805710345506668, validation loss : 0.658903181552887
Early stop counter: 4
Epoch: 6/300, train loss : 0.6755501329898834, validation loss : 0.7144209742546082
Early stop counter: 5
Epoch: 7/300, train loss : 0.680072546005249, validation loss : 0.6758288741111755
Early stop counter: 6
Epoch: 8/300, train loss : 0.6730537563562393, validation loss : 0.6894646286964417
Early stop counter: 7
Epoch: 9/300, train loss : 0.6720863878726959, validation loss : 0.6833751201629639
Early stop counter: 8
Epoch: 10/300, train loss : 0.66546277701

[I 2023-03-08 17:26:50,785] Trial 0 finished with value: 0.646298611164093 and parameters: {'num_layers': 2, 'hidden_size': 135, 'learning_rate': 0.00693648872676605}. Best is trial 0 with value: 0.646298611164093.


Epoch: 7/300, train loss : 0.6747991144657135, validation loss : 0.7073723673820496
Early stop counter: 6
Epoch: 8/300, train loss : 0.6734083294868469, validation loss : 0.697421133518219
Early stop counter: 7
Epoch: 9/300, train loss : 0.6735455095767975, validation loss : 0.7038393616676331
Early stop counter: 8
Epoch: 10/300, train loss : 0.6684256196022034, validation loss : 0.7008915543556213
Early stop counter: 9
Epoch: 11/300, train loss : 0.6659853756427765, validation loss : 0.700866162776947
Early stop counter: 10
Epoch: 12/300, train loss : 0.6623090207576752, validation loss : 0.7017859220504761
Early stopping...
Fold 0
Epoch: 1/300, train loss : 1.8657525181770325, validation loss : 0.6871082782745361
Early stop counter: 0
Epoch: 2/300, train loss : 0.6993994861841202, validation loss : 0.6894806027412415
Early stop counter: 1
Epoch: 3/300, train loss : 0.6888601183891296, validation loss : 0.7056416273117065
Early stop counter: 2
Epoch: 4/300, train loss : 0.695075169205

[I 2023-03-08 17:27:00,981] Trial 1 finished with value: 0.6417305946350098 and parameters: {'num_layers': 3, 'hidden_size': 236, 'learning_rate': 0.0029548898157339566}. Best is trial 1 with value: 0.6417305946350098.


...

Epoch: 13/300, train loss : 0.6234825849533081, validation loss : 0.7275334000587463
Early stop counter: 10
Epoch: 14/300, train loss : 0.6175742000341415, validation loss : 0.6916236281394958
Early stopping...
Fold 0
Epoch: 1/300, train loss : 1.4392645359039307, validation loss : 0.6363632082939148
Early stop counter: 0
Epoch: 2/300, train loss : 0.7332737296819687, validation loss : 0.7198876738548279
Early stop counter: 1
Epoch: 3/300, train loss : 0.6942693889141083, validation loss : 0.7095155119895935
Early stop counter: 2
Epoch: 4/300, train loss : 0.6912022829055786, validation loss : 0.690505862236023
Early stop counter: 3
Epoch: 5/300, train loss : 0.6890516877174377, validation loss : 0.7084537148475647
Early stop counter: 4
Epoch: 6/300, train loss : 0.6867453902959824, validation loss : 0.6814353466033936
Early stop counter: 5
Epoch: 7/300, train loss : 0.6775660812854767, validation loss : 0.716947615146637
Early stop counter: 6
Epoch: 8/300, train loss : 0.6824343949556

[I 2023-03-08 17:30:26,689] Trial 28 finished with value: 0.6440068125724793 and parameters: {'num_layers': 1, 'hidden_size': 99, 'learning_rate': 0.006318000724240208}. Best is trial 13 with value: 0.6263818264007568.


Epoch: 12/300, train loss : 0.6475618779659271, validation loss : 0.6905208230018616
Early stop counter: 10
Epoch: 13/300, train loss : 0.6425556987524033, validation loss : 0.6812742948532104
Early stopping...
Fold 0
Epoch: 1/300, train loss : 41.74121472239494, validation loss : 1.5528576374053955
Early stop counter: 0
Epoch: 2/300, train loss : 1.9624030590057373, validation loss : 0.6341509222984314
Early stop counter: 0
Epoch: 3/300, train loss : 0.7329131364822388, validation loss : 0.7390626072883606
Early stop counter: 1
Epoch: 4/300, train loss : 0.6985687762498856, validation loss : 0.6848932504653931
Early stop counter: 2
Epoch: 5/300, train loss : 0.6915372610092163, validation loss : 0.6954882740974426
Early stop counter: 3
Epoch: 6/300, train loss : 0.6900333166122437, validation loss : 0.7153339982032776
Early stop counter: 4
Epoch: 7/300, train loss : 0.6860326379537582, validation loss : 0.6842126250267029
Early stop counter: 5
Epoch: 8/300, train loss : 0.682306110858

[I 2023-03-08 17:30:32,719] Trial 29 finished with value: 0.65691579580307 and parameters: {'num_layers': 2, 'hidden_size': 427, 'learning_rate': 0.00667738334985686}. Best is trial 13 with value: 0.6263818264007568.


Epoch: 11/300, train loss : 0.6682745516300201, validation loss : 0.7099030613899231
Early stop counter: 8
Epoch: 12/300, train loss : 0.6685814410448074, validation loss : 0.7033895254135132
Early stop counter: 9
Epoch: 13/300, train loss : 0.6641942262649536, validation loss : 0.6898854970932007
Early stop counter: 10
Epoch: 14/300, train loss : 0.6578496545553207, validation loss : 0.6835047006607056
Early stopping...
best trial:
[0.6263818264007568]
Best parameters: {'num_layers': 1, 'hidden_size': 66, 'learning_rate': 0.00889495369073538}


## 3. Train/validate/test

1. After hyperparameters tuning, the best parameters saved to config.py as params_gin
2. Next, we will train/validate/test the model using 5-fold CV followed by repeating the process for 5 times.
3. Ensure that data folder, from_scratch_trained_models folder and this notebook are in the same working directory
4. run_training to repeat the training process, please create a new folder to retrain, save and load saved model
5. Else, comment the run_training function and do testing to get the results which were obtained in the paper. 

In [2]:
def run_training(train_loader, valid_loader, params, trained_model_path):
    model = GIN(num_features=NUM_FEATURES, num_targets=NUM_TARGET, num_layers=params['num_layers'], hidden_size=params['hidden_size'])
    model.to(DEVICE)
    optimizer=torch.optim.Adam(model.parameters(),lr = params['learning_rate'])
    eng = EngineHOB_no_edge(model, optimizer, device=DEVICE)

    best_loss = np.inf
    early_stopping_iter = PATIENCE
    early_stopping_counter = 0 

    for epoch in range(EPOCHS):
        train_loss= eng.train(train_loader)
        valid_loss= eng.validate(valid_loader)
        print(f'Epoch: {epoch+1}/{EPOCHS}, train loss : {train_loss}, validation loss : {valid_loss}')
        if valid_loss < best_loss:
            best_loss = valid_loss 
            early_stopping_counter=0 #reset counter
            print('Saving model...')
            torch.save(model.state_dict(), trained_model_path)
        else:
            early_stopping_counter +=1

        if early_stopping_counter > early_stopping_iter:
            print('Early stopping...')
            break
        print(f'Early stop counter: {early_stopping_counter}')
    
    return best_loss

def run_testing(test_loader, params, trained_model_path):
    model = GIN(num_features=NUM_FEATURES, num_targets=NUM_TARGET, num_layers=params['num_layers'], hidden_size=params['hidden_size'])
    model.load_state_dict(torch.load(trained_model_path))
    model.to(DEVICE)
    optimizer=torch.optim.Adam(model.parameters(),lr = params['learning_rate'])
    eng = EngineHOB_no_edge(model, optimizer, device=DEVICE)

    print('Begin testing...')
    bce, acc, f1, roc_auc= eng.test(test_loader)
    print('Test completed!')
    print(f'bce:{bce}, acc :{acc}, f1: {f1}, roc_auc: {roc_auc}')
    return bce, acc, f1, roc_auc

In [7]:
n_repetitions = 5
params = params_gin
train_data_root_path = './data/graph_data/data_oral_avail_train/'
train_data_raw_filename = 'data_oral_avail_train_50.csv'
test_data_root_path = './data/graph_data/data_oral_avail_test/'
test_data_raw_filename = 'data_oral_avail_test_1_50.csv'
path_to_save_trained_model = './from_scratch_trained_models/GIN/'

bce_list = []
acc_list = []
f1_list = []
roc_auc_list = []

#load dataset 
dataset_for_cv = LoadHOBDataset(root=train_data_root_path, raw_filename=train_data_raw_filename)
test_dataset = LoadHOBDataset(root=test_data_root_path, raw_filename=test_data_raw_filename)

kf = KFold(n_splits= N_SPLITS)

for repeat in range(n_repetitions):
    for fold_no, (train_idx, valid_idx) in enumerate(kf.split(dataset_for_cv)):
        print(f'For rep: {repeat}, fold: {fold_no}')
        seed_everything(SEED_NO)
        train_dataset= []
        valid_dataset = []
        for t_idx in train_idx:
            train_dataset.append(torch.load(f'./data/graph_data/data_oral_avail_train/processed/molecule_{t_idx}.pt'))
        for v_idx in valid_idx:
            valid_dataset.append(torch.load(f'./data/graph_data/data_oral_avail_train/processed/molecule_{v_idx}.pt'))

        train_loader = DataLoader(train_dataset, batch_size=NUM_GRAPHS_PER_BATCH, shuffle=True)
        valid_loader = DataLoader(valid_dataset, batch_size=NUM_GRAPHS_PER_BATCH, shuffle=False)
        test_loader = DataLoader(test_dataset, batch_size=NUM_GRAPHS_PER_BATCH, shuffle=False)

        run_training(train_loader, valid_loader, params, os.path.join(path_to_save_trained_model, f'gin_repeat_{repeat}_fold_{fold_no}.pt'))
        bce, acc, f1, roc_auc = run_testing(test_loader, params, os.path.join(path_to_save_trained_model, f'gin_repeat_{repeat}_fold_{fold_no}.pt'))
        bce_list.append(bce)
        acc_list.append(acc)
        f1_list.append(f1)
        roc_auc_list.append(roc_auc)

bce_arr = np.array(bce_list)
mean_bce = np.mean(bce_arr)
sd_bce = np.std(bce_arr)
print(f'bce:{mean_bce:.3f}±{sd_bce:.3f}')

acc_arr = np.array(acc_list)
acc_mean= np.mean(acc_arr)
acc_sd = np.std(acc_arr)
print(f'acc:{acc_mean:.3f}±{acc_sd:.3f}')

f1_arr = np.array(f1_list)
f1_mean= np.mean(f1_arr)
f1_sd = np.std(f1_arr)
print(f'f1: {f1_mean:.3f}±{f1_sd:.3f}')

roc_auc_arr = np.array(roc_auc_list)
roc_auc_mean= np.mean(roc_auc_arr)
roc_auc_sd = np.std(roc_auc_arr)
print(f'roc_auc: {roc_auc_mean:.3f}±{roc_auc_sd:.3f}')

For rep: 0, fold: 0
Epoch: 1/300, train loss : 1.4474536180496216, validation loss : 0.6468539237976074
Saving model...
Early stop counter: 0
Epoch: 2/300, train loss : 0.7944114208221436, validation loss : 0.7610831260681152
Early stop counter: 1
Epoch: 3/300, train loss : 0.7047408521175385, validation loss : 0.7292219996452332
Early stop counter: 2
Epoch: 4/300, train loss : 0.6896027475595474, validation loss : 0.7106878161430359
Early stop counter: 3
Epoch: 5/300, train loss : 0.686515599489212, validation loss : 0.6983362436294556
Early stop counter: 4
Epoch: 6/300, train loss : 0.6845705062150955, validation loss : 0.6978377103805542
Early stop counter: 5
Epoch: 7/300, train loss : 0.6809114813804626, validation loss : 0.7046172022819519
Early stop counter: 6
Epoch: 8/300, train loss : 0.6812890619039536, validation loss : 0.6997275948524475
Early stop counter: 7
Epoch: 9/300, train loss : 0.6773758679628372, validation loss : 0.6876591444015503
Early stop counter: 8
Epoch: 10/3